In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

# Loading dataset

In [2]:
from datasets import load_dataset, load_metric

In [3]:
datasets = load_dataset("conll2003")

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
datasets["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,chunk_tags,id,ner_tags,pos_tags,tokens
0,"[B-NP, I-NP]",4603,"[O, B-PER]","[CD, NNP]","[1., Tortelli]"
1,"[B-NP, I-NP, I-NP, O, B-NP, O]",13812,"[O, B-ORG, I-ORG, O, O, O]","[:, NNP, NNP, (, CD, )]","[--, Singapore, Newsroom, (, +65-8703086, )]"
2,"[B-NP, I-NP, B-VP, B-NP, I-NP, I-NP, B-NP, I-NP, I-NP, B-PP, B-NP, B-PP, I-PP, B-VP, B-NP, I-NP, I-NP, B-PP, B-NP, O]",10236,"[B-PER, I-PER, O, O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, O]","[NNP, NNP, VBD, PRP$, NN, NNP, POS, JJ, NN, IN, NNP, RB, IN, VBG, PRP$, NN, NN, IN, NNP, .]","[Liam, Botham, demonstrated, his, father, Ian, 's, golden, touch, on, Wednesday, shortly, after, making, his, county, debut, for, Hampshire, .]"
3,"[B-NP, B-PP, B-NP, I-NP, B-VP, B-PP, B-NP, B-NP, B-VP, I-VP, B-NP, I-NP, I-NP, I-NP, I-NP, O]",1696,"[B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O]","[NNP, IN, NNP, NNP, VBD, IN, NNP, PRP, VBD, VBG, PRP$, JJ, NN, NN, NNS, .]","[Bank, of, New, Zealand, said, on, Thursday, it, was, cutting, its, fixed, home, lending, rates, .]"
4,"[B-NP, B-VP, I-VP, I-VP, B-NP, I-NP, B-ADVP, B-PP, B-NP, I-NP, I-NP, O, B-VP, B-NP, B-VP, I-VP, I-VP, I-VP, I-VP, B-NP, I-NP, I-NP, O]",13002,"[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[NNP, VBD, TO, VB, DT, NNS, RB, IN, PRP$, NN, NN, CC, VBD, PRP, VBD, VBG, TO, VB, VBG, DT, NN, NN, .]","[Morris, declined, to, address, the, allegations, directly, in, his, resignation, statement, but, said, he, was, quitting, to, avoid, becoming, a, campaign, issue, .]"
5,"[B-PP, B-NP, I-NP, I-NP, O]",6809,"[O, O, B-MISC, O, O]","[IN, DT, JJ, NN, .]","[in, the, Argentine, season, .]"
6,"[B-NP, I-NP, I-NP, I-NP, B-VP, I-VP, B-PP, B-NP, B-PP, B-NP, I-NP, O, B-NP, I-NP, O, B-PP, B-NP, B-SBAR, B-NP, B-VP, B-NP, B-PP, B-NP, O, B-VP, B-NP, I-NP, O, B-NP, I-NP, O, B-NP, I-NP, B-VP, B-NP, I-NP, I-NP, B-PP, B-VP, O]",4265,"[O, O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[DT, CD, JJ, NNS, VBD, VBN, IN, NNP, IN, CD, NNS, (, CD, NNS, ), IN, NN, IN, PRP, VBD, NN, IN, NN, CC, VBD, DT, NN, CC, VBN, CD, ,, DT, NN, VBD, DT, NN, NN, IN, VBG, .]","[The, two, Iranian, men, were, arrested, in, July, with, 419, kilograms, (, 924, lbs, ), of, opium, after, they, opened, fire, on, police, and, killed, a, pedestrain, and, wounded, four, ,, the, newspaper, quoted, a, police, commander, as, saying, .]"
7,"[B-NP, B-VP, B-ADJP, B-NP, B-VP, I-VP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-VP, B-PP, B-NP, B-PP, B-NP, I-NP, O]",1458,"[B-PER, O, O, O, O, O, O, O, O, O, B-MISC, O, O, O, O, B-LOC, O, O, O, O]","[NN, VBZ, JJ, NN, VBZ, VBN, DT, JJ, NN, IN, JJ, NN, NN, VBG, IN, NNP, IN, DT, NNS, .]","[Lauck, 's, worldwide, network, has, been, the, main, source, of, anti-Semitic, propaganda, material, flowing, into, Germany, since, the, 1970s, .]"
8,"[O, B-PP, B-NP, I-NP, I-NP, I-NP, B-PP, B-VP, I-VP, O, B-NP, I-NP, B-PP, B-NP, B-VP, B-ADJP, I-ADJP, O]",4129,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O]","["", IN, DT, JJ, CD, NNS, IN, VBG, VBN, ,, DT, NN, IN, NNP, VBD, RB, JJ, .]","["", During, the, last, 15, days, of, being, held, ,, the, fighting, in, Grozny, was, very, close, .]"
9,"[B-NP, B-PP, B-NP, B-VP, B-NP, B-NP, I-NP, O, B-PP, B-NP, O, B-VP, B-NP, B-VP, I-VP, B-PP, B-NP, O, B-VP, I-VP, I-VP, B-NP, B-PP, B-NP, O]",102,"[B-LOC, O, O, O, B-LOC, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[NNP, IN, NNP, VBD, NNP, DT, NN, ,, IN, NNP, ,, VBG, PRP, VBD, VBN, TO, NN, CC, VBD, TO, VB, NNS, IN, NNS, .]","[Israel, on, Wednesday, sent, Syria, a, message, ,, via, Washington, ,, saying, it, was, committed, to, peace, and, wanted, to, open, negotiations, without, preconditions, .]"


# Preprocessing dataset

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [14]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [15]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [16]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [17]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


In [18]:
label_all_tokens = True

In [19]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [20]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

# Fine-tuning the model

In [22]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [23]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
metric = load_metric("seqeval")

In [28]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [29]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,0.251100,0.069681,0.919468,0.920908,0.920188,0.981206,1.741600,1866.113000
2,0.053300,0.062200,0.926305,0.930865,0.928579,0.983240,1.734500,1873.790000
3,0.030500,0.062679,0.924385,0.936794,0.930548,0.983669,1.781300,1824.498000


TrainOutput(global_step=2634, training_loss=0.08752527930289394, metrics={'train_runtime': 59.1619, 'train_samples_per_second': 44.522, 'total_flos': 795751480997262.0, 'epoch': 3.0})

In [32]:
trainer.evaluate()

{'eval_loss': 0.06267944723367691,
 'eval_precision': 0.9243845899105861,
 'eval_recall': 0.9367938248126189,
 'eval_f1': 0.9305478386487388,
 'eval_accuracy': 0.983668800737128,
 'eval_runtime': 1.7222,
 'eval_samples_per_second': 1887.17,
 'epoch': 3.0}

In [33]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.9506219374293253,
  'recall': 0.9633307868601986,
  'f1': 0.9569341680895466,
  'number': 2618},
 'MISC': {'precision': 0.8079937304075235,
  'recall': 0.8375304630381804,
  'f1': 0.8224970083765457,
  'number': 1231},
 'ORG': {'precision': 0.8892601431980907,
  'recall': 0.9061284046692607,
  'f1': 0.8976150325222837,
  'number': 2056},
 'PER': {'precision': 0.9746293245469522,
  'recall': 0.974950560316414,
  'f1': 0.9747899159663866,
  'number': 3034},
 'overall_precision': 0.9243845899105861,
 'overall_recall': 0.9367938248126189,
 'overall_f1': 0.9305478386487388,
 'overall_accuracy': 0.983668800737128}